### Try-it 9.2: Predicting Wages

This activity is meant to summarize your work with regularized regression models.  You will use your earlier work with data preparation and pipelines together with what you've learned with grid searches to determine an optimal model.  In addition to the prior strategies, this example is an excellent opportunity to utilize the `TransformedTargetRegressor` estimator in scikitlearn.

### The Data

This dataset is loaded from the openml resource library.  Originally from census data, the data contains wage and demographic information on 534 individuals. From the dataset documentation [here](https://www.openml.org/d/534)

```
The Current Population Survey (CPS) is used to supplement census information between census years. These data consist of a random sample of 534 persons from the CPS, with information on wages and other characteristics of the workers, including sex, number of years of education, years of work experience, occupational status, region of residence and union membership. 
```

In [134]:
from sklearn.datasets import fetch_openml

In [135]:
wages = fetch_openml(data_id=534, as_frame=True)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [136]:
wages.frame.head()

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,WAGE,AGE,RACE,OCCUPATION,SECTOR,MARR
0,8,no,female,21,not_member,5.10,35,Hispanic,Other,Manufacturing,Married
1,9,no,female,42,not_member,4.95,57,White,Other,Manufacturing,Married
2,12,no,male,1,not_member,6.67,19,White,Other,Manufacturing,Unmarried
3,12,no,male,4,not_member,4.00,22,White,Other,Other,Unmarried
4,12,no,male,17,not_member,7.50,35,White,Other,Other,Married


#### Task

Build regression models to predict `WAGE`.  Incorporate the categorical features and transform the target using a logarithm.  Build `Ridge` models and consider some different amounts of regularization.  

After fitting your model, interpret the model and try to understand what features led to higher wages.  Consider using `permutation_importance` that you encountered in module 8.  Discuss your findings in the class forum.

For an in depth example discussing the perils of interpreting the coefficients see the example in scikitlearn examples [here](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html).

In [137]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.express as px
import pandas as pd
import warnings
from sklearn.pipeline import make_pipeline
import numpy as np
import scipy as sp

In [138]:
wages_df = wages.frame

In [139]:
wages_df.shape

(534, 11)

In [140]:
wages_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   EDUCATION   534 non-null    int64   
 1   SOUTH       534 non-null    category
 2   SEX         534 non-null    category
 3   EXPERIENCE  534 non-null    int64   
 4   UNION       534 non-null    category
 5   WAGE        534 non-null    float64 
 6   AGE         534 non-null    int64   
 7   RACE        534 non-null    category
 8   OCCUPATION  534 non-null    category
 9   SECTOR      534 non-null    category
 10  MARR        534 non-null    category
dtypes: category(7), float64(1), int64(3)
memory usage: 21.4 KB


In [141]:
wages_df.corr()

/var/folders/dy/mqrlzklx5s17wl5wc4bfrw4w0000gp/T/ipykernel_27857/11099405.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  wages_df.corr()


,EDUCATION,EXPERIENCE,WAGE,AGE
EDUCATION,1.000000,-0.352676,0.381922,-0.150019
EXPERIENCE,-0.352676,1.000000,0.087060,0.977961
WAGE,0.381922,0.087060,1.000000,0.176967
AGE,-0.150019,0.977961,0.176967,1.000000


In [142]:
X = wages_df.drop("WAGE", axis=1)
y = wages_df['WAGE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1234)
categorical_columns = ["RACE", "OCCUPATION", "SECTOR", "MARR", "UNION", "SEX", "SOUTH"]
numerical_columns = ["EDUCATION", "EXPERIENCE", "AGE"]

In [143]:
col_transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), categorical_columns), 
                                          (PolynomialFeatures(include_bias = False), numerical_columns),
                                          remainder='passthrough')
col_transformer.fit_transform(X,y)

array([[1.000e+00, 0.000e+00, 0.000e+00, ..., 4.410e+02, 7.350e+02,
        1.225e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 1.764e+03, 2.394e+03,
        3.249e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 1.000e+00, 1.900e+01,
        3.610e+02],
       ...,
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 6.250e+02, 1.200e+03,
        2.304e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 1.690e+02, 4.030e+02,
        9.610e+02],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 1.089e+03, 1.815e+03,
        3.025e+03]])

In [144]:
pipe = Pipeline([
    ('ohe_transform', col_transformer),
    ('scaled', StandardScaler()),
    ('linereg', Ridge())
])

In [145]:
param_dict = {'ohe_transform__polynomialfeatures__degree': [2,3,4,5],
             'linereg__alpha': np.logspace(0, 10, 50)}

In [146]:
grid_search_obj = GridSearchCV(
    pipe,
    param_grid = param_dict,
    scoring = 'neg_mean_squared_error',
    cv = 5
)

In [147]:
grid_search_obj.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ohe_transform',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='if_binary'),
                                                                         ['RACE',
                                                                          'OCCUPATION',
                                                                          'SECTOR',
                                                                          'MARR',
                                                                          'UNION',
                                                                          'SEX',
                                                                          'SOUTH']),
                                                                        ('polynomialfeatures',
                                                                         PolynomialFeatures(include_bias=False),
                                                                         ['EDUCATION',
                                                                          'EXPERIENCE',
                                                                          'AGE'])])),
                                       ('scaled', StandardScaler(...
       3.39322177e+06, 5.42867544e+06, 8.68511374e+06, 1.38949549e+07,
       2.22299648e+07, 3.55648031e+07, 5.68986603e+07, 9.10298178e+07,
       1.45634848e+08, 2.32995181e+08, 3.72759372e+08, 5.96362332e+08,
       9.54095476e+08, 1.52641797e+09, 2.44205309e+09, 3.90693994e+09,
       6.25055193e+09, 1.00000000e+10]),
                         'ohe_transform__polynomialfeatures__degree': [2, 3, 4,
                                                                       5]},
             scoring='neg_mean_squared_error')

In [148]:
best_model = grid_search_obj.best_estimator_
test_preds = best_model.predict(X_test)
test_mse = mean_squared_error(test_preds, y_test)
print(test_mse)

15.262472715008938


In [149]:
preprocessor = make_column_transformer(
    (OneHotEncoder(drop="if_binary"), categorical_columns),
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

In [150]:
model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Ridge(alpha=1e-10), func=np.log10, inverse_func=sp.special.exp10
    ),
)

In [151]:
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['RACE', 'OCCUPATION',
                                                   'SECTOR', 'MARR', 'UNION',
                                                   'SEX', 'SOUTH'])],
                                   verbose_feature_names_out=False)),
                ('transformedtargetregressor',
                 TransformedTargetRegressor(func=<ufunc 'log10'>,
                                            inverse_func=<ufunc 'exp10'>,
                                            regressor=Ridge(alpha=1e-10)))])

In [152]:
test_preds = model.predict(X_test)
test_mse = mean_squared_error(test_preds, y_test)
print(test_mse)

16.447500541486644
